# Aethr - YOLO11 Training (Google Colab T4)\n\nUse this notebook with the **VS Code Google Colab extension**.\nSet runtime to **GPU (T4)** before training.

In [ ]:
from pathlib import Path\nimport os\nimport shutil\n\nPROJECT_ROOT = Path('/content/aethr')\nprint(f'PROJECT_ROOT exists: {PROJECT_ROOT.exists()}')\nif PROJECT_ROOT.exists():\n    os.chdir(PROJECT_ROOT)\n\n!nvidia-smi\n!pip install -q -r requirements.txt

In [ ]:
from pathlib import Path\n\ndef find_data_yaml(base: Path) -> Path:\n    candidates = sorted(base.rglob('data.yaml'))\n    if not candidates:\n        raise FileNotFoundError('No data.yaml found under data/raw')\n    return candidates[0]\n\ndata_yaml = find_data_yaml(PROJECT_ROOT / 'data' / 'raw')\nprint('Using:', data_yaml)

In [ ]:
from ultralytics import YOLO\n\nmodel = YOLO('yolo11s.pt')\nresults = model.train(\n    data=str(data_yaml),\n    epochs=80,\n    imgsz=640,\n    batch=16,\n    project=str(PROJECT_ROOT / 'runs'),\n    name='aethr_colab',\n    device=0\n)\nprint('Training finished. Save dir:', results.save_dir)

In [ ]:
from pathlib import Path\nimport shutil\nfrom ultralytics import YOLO\n\nsave_dir = Path(results.save_dir)\nbest_pt = save_dir / 'weights' / 'best.pt'\ndest_pt = PROJECT_ROOT / 'models' / 'best.pt'\ndest_pt.parent.mkdir(parents=True, exist_ok=True)\nshutil.copy2(best_pt, dest_pt)\nprint('Copied best model to', dest_pt)\n\nexport_model = YOLO(str(dest_pt))\nexport_model.export(format='onnx', imgsz=640)\nprint('ONNX export complete. Check models/ and runs/ directories.')